# PyMySQL再封装

In [1]:
import pymysql
import pandas as pd

pd.set_option("display.max_columns", 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option("expand_frame_repr", False)  # 当列太多时不自动换行
pd.set_option('max_colwidth', 255)  # 单元格最大数据长度


class MysqlDB(object):
    def __init__(self, db='nba'):
        self.__db = db

    def __load_config(self):
        with open("config.json") as config:
            return json.load(config) 
        
        
    def __connect(self):
        config = self.__load_config()
        return pymysql.connect(host=config["host"],
                               user=config["user"],
                               password=config["password"],
                               db=self.__db,
                               charset='utf8mb4',
                               cursorclass=pymysql.cursors.DictCursor)

    def db_query(self, sql, args=None):
        """
        sql: str, sql语句
        args（元组，列表或字典） - 与查询一起使用的参数。
        输出sql查询结果(tuple)
        """
        conn = self.__connect()
        try:
            with conn.cursor() as cursor:
                rows = cursor.execute(sql, args)  # 受影响的行数
                res = cursor.fetchall()
                columns = [column[0] for column in cursor.description]
                if rows == 0:
                    return None
                return f'Affected {rows} rows', columns, res
        except Exception as e:
            return {"error": f'Exception error: {e}'}
        finally:
            conn.close()

    def db_query_df(self, sql, args=None):
        """
        sql: sql语句
        args（元组，列表或字典） - 与查询一起使用的参数。
        查询结果输出为pandas.DataFrame
        """
        res = self.db_query(sql, args)
        if res is None:
            return None
        if type(res) == 'dict':
            return res['error']
        try:
            df = pd.DataFrame(list(res[2]), columns=res[1])
            return df
        except Exception as e:
            return {"error": f'Exception error: {e}'}

    def db_execute_one(self, sql, args=None):
        """
        执行单条sql语句
        args（元组，列表或字典） - 与查询一起使用的参数。
        """
        conn = self.__connect()
        try:
            with conn.cursor() as cursor:
                rows = cursor.execute(sql, args)
            conn.commit()
            return f'Affected {rows} rows'
        except Exception as e:
            conn.rollback()
            return {"error": f"Exception error: {e}"}
        finally:
            conn.close()

    def db_execute_many(self, sql, args=None):
        """
        执行多条sql语句
        args（元组，列表或字典） - 与查询一起使用的参数。
        """
        conn = self.__connect()
        try:
            with conn.cursor() as cursor:
                rows = cursor.executemany(sql, args)
            conn.commit()
            return f'Affected {rows} rows'
        except Exception as e:
            conn.rollback()
            return {"error": f'Exception error: {e}'}
        finally:
            conn.close()

    def db_insert_one(self, sql, args=None):
        """
        插入单条数据
        args（元组，列表或字典） - 与查询一起使用的参数。
        """
        conn = self.__connect()
        try:
            with conn.cursor() as cursor:
                cursor.execute(sql, args)
                insert_id = cursor.lastrowid
            conn.commit()
            return insert_id
        except Exception as e:
            conn.rollback()
            return {"error": f'Exception error: {e}'}
        finally:
            conn.close()

    def db_insert_many(self, sql, args=None):
        """
        插入多条数据
        """
        conn = self.__connect()
        try:
            with conn.cursor() as cursor:
                rows = cursor.executemany(sql, args)
                insert_id = cursor.lastrowid
            conn.commit()
            return insert_id, insert_id + len(
                args) - 1, f'Affected {rows} rows'
        except Exception as e:
            conn.rollback()
            return {"error": f'Exception error: {e}'}
        finally:
            conn.close()

    def db_callproc(self, sql, arg=()):
            """
            执行存储过程
            """
            conn = self.__connect()
            try:
                with conn.cursor() as cursor:
                    return cursor.callproc(sql, arg)
            except Exception as e:
                conn.rollback()
                return {"error": f'Exception error: {e}'}
            finally:
                conn.close()


db = MysqlDB()

# 增

## 新增单行

In [2]:
# db = MysqlDB()
sql = 'INSERT INTO player (team_id, player_name, height) VALUES ("1003","科比-布莱恩特", 1.98);'
sql = 'INSERT INTO player (team_id, player_name, height) VALUES (%s,%s,%s);'

team_id = '1003'
player_name = '科比-布莱恩特'
height = 2.00

fields_list = [team_id, player_name, height]
db.db_insert_one(sql, fields_list)

10186

## 新增多行

In [3]:
# db = MysqlDB()
sql = 'INSERT INTO player (team_id, player_name, height) VALUES (%s,%s,%s);'
data = [(1003, '勒布朗-詹姆斯', 2.02), (1004, '迈克尔-乔丹', 1.98), (1005, '姚明', 2.26)]
db.db_insert_many(sql, data)

(10187, 10189, 'Affected 3 rows')

**主要用到`executemany`这个方法**

`executemany`（**query**，*args* ）

针对一个查询运行多个数据

| 参数： | 1. **query** - 要在服务器上执行的查询 2. **args** - 序列或映射的序列(元组，列表或字典)。用作参数。 |
| :----- | ------------------------------------------------------------ |
| 返回： | 受影响的行数（如果有）。                                     |

此方法可提高多行INSERT和REPLACE的性能。否则它等同于使用execute()循环遍历args。

In [4]:
sql = 'INSERT INTO player (team_id, player_name, height) VALUES (%s,%s,%s);'
data = [(1003, '勒布朗-詹姆斯', 2.02), (1004, '迈克尔-乔丹', 1.98), (1005, '姚明', 2.26)]
for i in range(len(data)):
    print(sql % (data[i]))


INSERT INTO player (team_id, player_name, height) VALUES (1003,勒布朗-詹姆斯,2.02);
INSERT INTO player (team_id, player_name, height) VALUES (1004,迈克尔-乔丹,1.98);
INSERT INTO player (team_id, player_name, height) VALUES (1005,姚明,2.26);


In [5]:
args = ['@max_max_hp', '@min_max_mp', '@avg_max_attack']
# proc = args.append('战士')
sql = "select %s, %s, %s;"
print(sql % tuple(args))

select @max_max_hp, @min_max_mp, @avg_max_attack;


# 删/改
删改操作使用`db_execute_one()`或者`db_execute_many()`方法即可

## 修改单行

In [6]:
# db = MysqlDB()
sql = "UPDATE player SET height=%s WHERE player_name=%s;"
update = [2.03, "勒布朗-詹姆斯"]
db.db_execute_one(sql, update)

'Affected 1 rows'

## 修改多行

In [7]:
# db = MysqlDB()
sql = "UPDATE player SET height=%s WHERE player_name=%s;"
update = [(2.04, "勒布朗-詹姆斯"), (1.99, '迈克尔-乔丹'), (2.27, '姚明')]
db.db_execute_many(sql, update)

'Affected 3 rows'

## 删除单行

In [8]:
# db = MysqlDB()
sql = "DELETE FROM player WHERE player_name=%s;"
player_name = "科比-布莱恩特"
db.db_execute_one(sql, player_name)

'Affected 1 rows'

## 删除多行

In [9]:
# db = MysqlDB()
sql = "DELETE FROM player WHERE player_name = %s;"
player_names = ['勒布朗-詹姆斯', '迈克尔-乔丹', '姚明']
db.db_execute_many(sql, player_names)

'Affected 3 rows'

# 查

## 返回元组

In [10]:
# db = MysqlDB()
sql = "SELECT * FROM nba.player WHERE height = %s AND team_id = %s LIMIT 5;"
arg = [1.93, '1003']
print(db.db_query(sql, arg))

None


In [27]:
# db = MysqlDB()
sql = "SELECT * FROM nba.player WHERE player_id = %s LIMIT 5;"
arg = ['10001']
db.db_query(sql, arg)

('Affected 1 rows',
 ['player_id', 'team_id', 'player_name', 'height'],
 [{'player_id': 10001,
   'team_id': 1001,
   'player_name': '韦恩-艾灵顿',
   'height': 1.93}])

## 返回pandas.DataFrame

In [23]:
# db = MysqlDB()
sql = "SELECT * FROM nba.player WHERE height = %s AND team_id = %s LIMIT 5;"
arg = [1.93, '1001']
db.db_query_df(sql, arg)

,player_id,team_id,player_name,height
0,10001,1001,韦恩-艾灵顿,1.93
1,10019,1001,扎克-洛夫顿,1.93


In [24]:
# db = MysqlDB()
sql = "SELECT * FROM nba.player WHERE height = %s AND team_id = %s LIMIT 5;"
arg = [1.93, '1003']
print(db.db_query_df(sql, arg))

None
